In [17]:
import glob

# Define your directory
fil_dir = "/home/leeeefun681/volume/eefun/webscraping/sitemap/sitemap_scrape/data/business_times/unsuccessful"

# Use glob to find all .txt files in the directory
txt_files = glob.glob(f"{fil_dir}/*.jsonl")

# Initialize a counter
total_lines = 0

# Loop through each file and count lines
for file_path in txt_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        total_lines += len(lines)

print(f"Total URL lines in directory: {total_lines}")
print(f"Average error rate for each file: {total_lines/len(txt_files)}")


Total URL lines in directory: 163
Average error rate for each file: 1.147887323943662


In [12]:
import glob
import json

# directory containing your .jsonl files
fil_dir = "/home/leeeefun681/volume/eefun/webscraping/sitemap/sitemap_scrape/data/business_times/scraped"

# grab all .jsonl paths
jsonl_files = glob.glob(f"{fil_dir}/*.jsonl")

total_images = 0
for file_path in jsonl_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                obj = json.loads(line)
            except json.JSONDecodeError:
                # skip any malformed lines
                continue
            imgs = obj.get("images", [])
            # make sure it's a list
            if isinstance(imgs, list):
                total_images += len(imgs)

print(f"Total images across all files: {total_images}")


Total images across all files: 278246


In [13]:
import glob
import json

# directory containing your .jsonl files
fil_dir = "/home/leeeefun681/volume/eefun/webscraping/sitemap/sitemap_scrape/data/straits_times/scraped"

# grab all .jsonl paths
jsonl_files = glob.glob(f"{fil_dir}/*.jsonl")

total_images = 0
for file_path in jsonl_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                obj = json.loads(line)
            except json.JSONDecodeError:
                # skip any malformed lines
                continue
            imgs = obj.get("images", [])
            # make sure it's a list
            if isinstance(imgs, list):
                total_images += len(imgs)

print(f"Total images across all files: {total_images}")


Total images across all files: 1742061
